In [ ]:
#export
from local.imports import *
from local.test import *
from local.core import *
from local.data.core import *
from local.data.external import *
from local.notebook.showdoc import show_doc

In [ ]:
#default_exp text.core

# Text core

> Basic function to preprocess text before assembling it in a `DataBunch`.

In [ ]:
#export 
from multiprocessing import Process, Queue
import spacy,html
from spacy.symbols import ORTH

## Preprocessing rules

The following are rules applied to texts before or after it's tokenized.

In [ ]:
#export
#special tokens
UNK, PAD, BOS, EOS, FLD, TK_REP, TK_WREP, TK_UP, TK_MAJ = "xxunk xxpad xxbos xxeos xxfld xxrep xxwrep xxup xxmaj".split()

In [ ]:
#export
_re_spec = re.compile(r'([/#\\])')

def spec_add_spaces(t):
    "Add spaces around / and #"
    return _re_spec.sub(r' \1 ', t)

In [ ]:
test_eq(spec_add_spaces('#fastai'), ' # fastai')
test_eq(spec_add_spaces('/fastai'), ' / fastai')
test_eq(spec_add_spaces('\\fastai'), ' \\ fastai')

In [ ]:
#export
_re_space = re.compile(' {2,}')

def rm_useless_spaces(t):
    "Remove multiple spaces"
    return _re_space.sub(' ', t)

In [ ]:
test_eq(rm_useless_spaces('a  b   c'), 'a b c')

In [ ]:
#export
_re_rep = re.compile(r'(\S)(\1{3,})')

def replace_rep(t):
    "Replace repetitions at the character level: cccc -> TK_REP 4 c"
    def _replace_rep(m):
        c,cc = m.groups()
        return f' {TK_REP} {len(cc)+1} {c} '
    return _re_rep.sub(_replace_rep, t)

It starts replacing at 4 repetitions of the same character or more.

In [ ]:
test_eq(replace_rep('aaa'), 'aaa')
test_eq(replace_rep('aaaa'), f' {TK_REP} 4 a ')

In [ ]:
#export
_re_wrep = re.compile(r'(?:\s|^)(\w+)\s+((?:\1\s+){2,})\1(\s|\W|$)')

#hide

Matches any word repeated at least four times with spaces between them
```
(?:\s|^)          Non-catching group with either a whitespace character or the beginning of text
(\w+)             Catching group of any alphanumeric character
\s+               One or more whitespace
((?:\1\s+){2,})   Catching group of a repetition of two or more times \1 followed by one or more whitespace
\1                Occurence of \1
(\s|\W|$)         Catching group of last whitespace, non alphanumeric character or end of text
```

In [ ]:
#export
def replace_wrep(t):
    "Replace word repetitions: word word word word -> TK_WREP 4 word"
    def _replace_wrep(m):
        c,cc,e = m.groups()
        return f' {TK_WREP} {len(cc.split())+2} {c} {e}'
    return _re_wrep.sub(_replace_wrep, t)

It starts replacing at 4 repetitions of the same word or more.

In [ ]:
test_eq(replace_wrep('ah ah'), 'ah ah')
test_eq(replace_wrep('ah ah ah ah'), f' {TK_WREP} 4 ah ')
test_eq(replace_wrep('ah ah   ah  ah'), f' {TK_WREP} 4 ah ')
test_eq(replace_wrep('ah ah ah ah '), f' {TK_WREP} 4 ah  ')
test_eq(replace_wrep('ah ah ah ah.'), f' {TK_WREP} 4 ah .')
test_eq(replace_wrep('ah ah ah ahi'), f'ah ah ah ahi')

In [ ]:
#export
def fix_html(x):
    "Various messy things we've seen in documents"
    x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace('nbsp;', ' ').replace(
        '#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace('<br />', "\n").replace(
        '\\"', '"').replace('<unk>',UNK).replace(' @.@ ','.').replace(' @-@ ','-')
    return html.unescape(x)

In [ ]:
test_eq(fix_html('#39;bli#146;'), "'bli'")
test_eq(fix_html('Sarah amp; Duck'), 'Sarah & Duck')
test_eq(fix_html('a nbsp; #36;'), 'a   $')
test_eq(fix_html('\\" <unk>'), f'" {UNK}')
test_eq(fix_html('quot;  @.@  @-@ '), "' .-")
test_eq(fix_html('<br />text\\n'), '\ntext\n')

In [ ]:
#export
_re_all_caps = re.compile(r'(\s|^)([A-Z]+[^a-z\s]*)(?=(\s|$))')

#hide

Catches any word in all caps, even with ' or - inside
```
(\s|^)        Catching group with either a whitespace or the beginning of text
([A-Z]+       Catching group with one capitalized letter or more...
[^a-z\s]*)    ...followed by anything that's non lowercase or whitespace
(?=(\s|$))    Look ahead for a space of end of text
```
The look ahead is there to not move the pointer ahead of the next space in case we have consecutive words in all caps.

In [ ]:
#export
def replace_all_caps(t):
    "Replace tokens in ALL CAPS by their lower version and add `TK_UP` before."
    def _replace_all_caps(m):
        tok = f'{TK_UP} ' if len(m.groups()[1]) > 1 else ''
        return f"{m.groups()[0]}{tok}{m.groups()[1].lower()}"
    return _re_all_caps.sub(_replace_all_caps, t)

In [ ]:
test_eq(replace_all_caps("I'M SHOUTING"), f"{TK_UP} i'm {TK_UP} shouting")
test_eq(replace_all_caps("I'm speaking normally"), "I'm speaking normally")
test_eq(replace_all_caps("I am speaking normally"), "i am speaking normally")

In [ ]:
#export
_re_maj = re.compile(r'(\s|^)([A-Z][^A-Z\s]*)(?=(\s|$))')

#hide

Catches any capitalized word
```
(\s|^)       Catching group with either a whitespace or the beginning of text
([A-Z]       Catching group with exactly one capitalized letter...
[^A-Z\s]*)   ...followed by anything that's not uppercase or whitespace
(?=(\s|$))   Look ahead for a space of end of text
```
The look ahead is there to not move the pointer ahead of the next space in case we have consecutive words in all caps.

In [ ]:
#export
def replace_maj(t):
    "Replace tokens in ALL CAPS by their lower version and add `TK_UP` before."
    def _replace_maj(m):
        tok = f'{TK_MAJ} ' if len(m.groups()[1]) > 1 else ''
        return f"{m.groups()[0]}{tok}{m.groups()[1].lower()}"
    return _re_maj.sub(_replace_maj, t)

In [ ]:
test_eq(replace_maj("Jeremy Howard"), f'{TK_MAJ} jeremy {TK_MAJ} howard')
test_eq(replace_maj("I don't think there is any maj here"), ("i don't think there is any maj here"),)

In [ ]:
#export
def lowercase(t):
    "Converts `t` to lowercase"
    return t.lower()

In [ ]:
defaults.text_proc_rules = [fix_html, replace_rep, replace_wrep, spec_add_spaces, rm_useless_spaces,
                            replace_all_caps, replace_maj, lowercase]